In [107]:
from rdflib import *
from rdflib.graph import Graph

In [108]:
modelet1 = Graph()
modelet1.parse("C:/Users/ghiot/Documents/TESI/last_onts/meaw_fin.ttl", format="ttl")

<Graph identifier=N4c7562aeceba43b98475c4126ee8d788 (<class 'rdflib.graph.Graph'>)>

In [109]:
# I think this is not working cause I still get : and no other namespace
ns = "https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#"
meaw = Namespace(ns)
meaw

Namespace('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#')

In [110]:
meaw = Namespace("https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
dul = Namespace("http://www.loa-cnr.it/ontologies/DUL.owl#")
modelet1.bind("meaw", meaw)
modelet1.bind("rdfs", rdfs)
modelet1.bind("owl", owl)
modelet1.bind("rdf", rdf)
modelet1.bind("xsd", xsd)
modelet1.bind("dul", dul)

In [ ]:
# checking triples

for s,p,o in modelet1.triples((None, rdf.type, None)):
    print(s, p, o)

In [ ]:
print(modelet1.serialize(format="ttl"))

### Competency questions

#### 1. Who participates in the talk show Welles Nietes?

Expected result:

The Id of a Person and, if named, their name.

* 358uU9uAvl), Legien Kromkamp
* Zqlw6RZQeX
* iBhFqi9HqQ, Adelheid Roosen

In [113]:
cq1 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
    SELECT ?agent ?name 
    WHERE {
        ?ts :involvesAgent ?agent;
            :hasTitle "Welles Nietes".
        ?agent a :Person.
        OPTIONAL{
            ?agent :hasName ?name
        }

    }


"""

results = modelet1.query(cq1)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#Zqlw6RZQeX'), None)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#iBhFqi9HqQ'), rdflib.term.Literal('Adelheid Roosen'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#358uU9uAvI'), rdflib.term.Literal('Legien Kromkamp'))




#### 2. Who is the person who appears to be expressing more emotions? What is their role?

Expected result: 

A list of people, if present their name, their roles, and the number of utterances in which they are expressing emotions. Sorted in descending order.

* 358uU9uAvl), Legien Kromkamp, H, 1
* Zqlw6RZQeX, SA, 3
* iBhFqi9HqQ, Adelheid Roosen. G, 5

In [114]:
cq2 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
    SELECT ?agent ?name ?role (COUNT(?emo) AS ?emo_count)
    WHERE {
        ?agent :hasConversationRolePlay ?conv_role;
                :isExperiencerIn ?emo_ann;
               a :Person.
        OPTIONAL{
            ?agent :hasName ?name
        }
        ?conv_role :hasRole ?role;
                     :hasConversationContext ?ts.
        ?ts :hasTitle "Welles Nietes".
        ?emo_ann :be_emotion ?emo.
        
    }

    GROUP BY ?agent
    ORDER BY DESC(?emo_count)
"""

results = modelet1.query(cq2)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#iBhFqi9HqQ'), rdflib.term.Literal('Adelheid Roosen'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#G'), rdflib.term.Literal('5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#Zqlw6RZQeX'), None, rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#SA'), rdflib.term.Literal('3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#358uU9uAvI'), rdflib.term.Literal('Legien Kromkamp'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#H'), rdflib.term.Literal('

#### 3. What are the conversational contexts identified for the utterance starting at 548 seconds in talk show Welles Nietes, and who is the agent?

Expected result: 

A list of conversational frames (id and name) the role of the agent.
* 3, argumenting_with_personal_experience, Guest

In [115]:
cq3 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?c_id ?c ?role
    WHERE {
        ?emo_ann :hasStart "548";
                :hasContext ?c_id;
                :experiencer ?agent.
        ?c_id rdfs:comment ?c.
        ?agent :hasConversationRolePlay ?rp.
        ?rp :hasRole ?role.        
    }
    """
results = modelet1.query(cq3)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#3'), rdflib.term.Literal('argumenting_with_personal_experience'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#G'))




#### 4. What are the utterances and conversational frames where Adelheid Rosen is angry? What is her emotion state?

Expected result: 

A list of utterances Id with related conversational context and emotion state annotation.
* 2101608040029452931_15_49, disagreeing with opinion, frustration
* 2101608040029452931_15_50, disagreeing with opinion, frustration

In [116]:
cq4 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?emo_ann_id ?c_name ?es
    WHERE {
        ?emo_ann_id :be_emotion :anger;
                :es_emotion ?es;
            :hasContext ?c;
            :experiencer ?agent.
        ?agent :hasName "Adelheid Roosen".
        ?c rdfs:comment ?c_name.
    }
    """
results = modelet1.query(cq4)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_49'), rdflib.term.Literal('disagreeing with opinion'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frustration'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_50'), rdflib.term.Literal('disagreeing with opinion'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frustration'))




#### 5. What is the topic of the conversation and what is the talk show with id 2101608050034534331, what is the talk show name? who is the broadcaster and what is the broadcasting date? How many utterances are annotated?

Expected result: 

The name of the talk show, the broadcaster’s name and the broadcasting date, a list of topics, the number of related emotion annotations.

* Het Blauwe Licht, VPRO, 17-03-1999, transgender_people, transitioning, religion,  4.

In [117]:
cq5 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?name ?broadcaster ?date ?topic (COUNT(?emo_ann) AS ?emo_count)
    WHERE {
        ?ts :hasIdentifier "2101608050034534331";
                             :hasTitle ?name;
                            :broadcastedBy ?broadcaster;
                            :hasDate ?date;
                            :isAbout ?topic;
                            dul:isSettingFor ?emo_ann
    }
        GROUP BY ?topic
    """
results = modelet1.query(cq5)

for row in results:
    print(row)
    print("\n")

(rdflib.term.Literal('Het Blauwe Licht'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#VPRO'), rdflib.term.Literal('17-03-1999'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#religion'), rdflib.term.Literal('4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.Literal('Het Blauwe Licht'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#VPRO'), rdflib.term.Literal('17-03-1999'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#transgender_people'), rdflib.term.Literal('4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.Literal('Het Blauwe Licht'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#VPRO'), rdflib.term.Literal

#### 6. What is the count of utterances that express anger, grouped by gender?

Expected result: 

The gender of the speaker and the number of the utterances in which they are angry

* female, 2

In [118]:
cq6 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?gender (COUNT(?emo_ann) AS ?count)
    WHERE {
        ?emo_ann :be_emotion :anger;
                :experiencer ?agent.
        ?agent :hasGender ?gender.  
    }
    GROUP BY ?gender
     
    """
results = modelet1.query(cq6)

for row in results:
    print(row)
    print("\n")


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#F'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))




#### 7. What is the number of utterances per emotion?

Expected result:

A basic emotion and the count of utterances where it’s expressed.

* sadness, 3
* fear, 2
* enjoyment, 5
* anger, 2
* surprise, 2

In [119]:
cq7 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?emo (COUNT(?emo_ann) AS ?count)
    WHERE {
        ?emo_ann :be_emotion ?emo.  
    }
    GROUP BY ?emo
     
    """
results = modelet1.query(cq7)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sadness'), rdflib.term.Literal('3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#fear'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#enjoyment'), rdflib.term.Literal('5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#anger'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#surprise'), rdflib.term.L

#### 8. What words triggers emotions?  

Expected result: 

A list of verbal triggers and respective basic emotion

* dat_ik_mij_dus_diep_en_diep_gekwetst_voelt, sadness
* en_zwaar_beledigd, sadness
* gekwetst, sadness
* gekwetst_zijn, sadness
* het_is_heel_vreemd_gegeven, surprise
* ik_wil_bijna_met_u_op_de_vuist_daarom, anger
* k_ben_doodzenuwachtig, fear
* k_denk_dus_dat_er_een_heleboel_mensen, sadness
* kwijt_hun_werk_kwijt_alles_kwijt_vrienden, fear
* leuk, enjoyment
* want_het_rare_is, surprise

In [120]:
cq8 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?trigger ?be
    WHERE {
        ?emo_ann :be_emotion ?be;
                :hasVerbalTrigger ?trigger  
    }
     
    """
results = modelet1.query(cq8)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#gekwetst'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sadness'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#gekwetst_zijn'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sadness'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#k_denk_dus_dat_er_een_heleboel_mensen'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sadness'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#dat_ik_mij_dus_diep_en_diep_gekwetst_voelt'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#s

#### 9. What are the utterances where triggers are visual?  

Expected result: 

The id of the utterance, the visual triggers

-	2101608040029452931_15_41, headshake
-	2101608040029452931_15_43, frown
-	2101608040029452931_15_44, look_at_interlocutor, raise eyebrows
-	2101608040029452931_15_46, hand to mouth, raise finger, smile
-	2101608040029452931_15_47, frown
-	2101608040029452931_15_48, frown, close eyes, hand to chest
-	2101608040029452931_15_49, raise finger
-	2101608040029452931_15_50, raise finger, frown
-	101608040029452931_15_51, raise finger, frown, raise inner brow, raise shoulder, raise upper body, stretch mouth, widen eyes
-	2101608040029452931_15_53, leaning forward

In [121]:
cq9 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?emo_ann ?trigger 
    WHERE {
        ?emo_ann :hasVisualTrigger ?trigger
    }
     
    """
results = modelet1.query(cq9)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_41'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#headshake'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_43'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frown'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_48'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frown'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_50'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#fro

#### 10. What are the utterances where the triggers are aural? 

Expected result: 

The id of the utterance, the aural triggers

* 2101608050034534331_9_137, intonation



In [122]:
cq10 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?emo_ann ?trigger 
    WHERE {
        ?emo_ann :hasAuralTrigger ?trigger
    }
     
    """
results = modelet1.query(cq10)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608050034534331_9_137'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#intonation'))




#### 11. What are the emotion states annotated for utterances that express sadness?

Expected results: The utterance text, the emotion states

-	Disappointment: Na jaren gekwetst door de laatste uitzending van adelheid roosen over de transseksuele ik vond dat het in het belachelijke getrokken werd en ik denk dus dat er een heleboel mensen daardoor gekwetst zijn want werd in het seksuele vlak getrokken en dat vond ik dus niet goed.
-	Sorrow: Misschien geïnterpreteerd als kwetsend nou ingewikkelde bij zo'n transseksueel is is is dat ik mij dus diep en diep gekwetst voelt en zwaar beledigd om het emotionele af dat mensen die dat uit hebben gezien mij beschuldigen van kwetsuren.
-	Disappointment: Het moet kiezen voor de persoon in kwestie of voor degenen die de talkshow leidt dat ik dus echt met alle respect wie acht minuten aan die persoon wilde geven als mensen dan kwetsend als kwetsend ervaren dan ben ik dus nou daar weer gekwetst en dan houdt het verhaal van mij op het uefa niets meer.
-	Sorrow: Het moet kiezen voor de persoon in kwestie of voor degenen die de talkshow leidt dat ik dus echt met alle respect wie acht minuten aan die persoon wilde geven als mensen dan kwetsend als kwetsend ervaren dan ben ik dus nou daar weer gekwetst en dan houdt het verhaal van mij op het uefa niets meer.


In [123]:
cq11 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?es ?text
    WHERE {
        ?emo_ann :be_emotion :sadness;
                :es_emotion ?es;
                :hasText ?text.
    }
     
    """
results = modelet1.query(cq11)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#disappointment'), rdflib.term.Literal('Na jaren gekwetst door de laatste uitzending van adelheid roosen over de transseksuele ik vond dat het in het belachelijke getrokken werd en ik denk dus dat er een heleboel mensen daardoor gekwetst zijn want werd in het seksuele vlak getrokken en dat vond ik dus niet goed.'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sorrow'), rdflib.term.Literal("Misschien geïnterpreteerd als kwetsend nou ingewikkelde bij zo'n transseksueel is is is dat ik mij dus diep en diep gekwetst voelt en zwaar beledigd om het emotionele af dat mensen die dat uit hebben gezien mij beschuldigen van kwetsuren."))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#disappointment'), rdflib.term.Literal('Het moet kiezen voor de persoon in kwestie of voor

#### 12. What utterances have a camera framing interpretation? What is the basic emotion?

Expected results:

utterance id, framing interpretation, basic emotion



In [124]:
cq12 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    
    SELECT ?emo_ann ?fi ?be

    WHERE{
    ?emo_ann :hasCameraFramingInterpretation ?fi;
            :be_emotion ?be
          
}

    """
results = modelet1.query(cq12)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_41'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#engaging_with'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#sadness'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_43'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#involvement'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#fear'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_46'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#contextualizing

#### 13. What are the types of frames that compose utterances where anger is shown?

Expected results:

utterance id, reference of shot, type of shot

-	2101608040029452931_15_49, 0, close up
-	2101608040029452931_15_49, 1, medium close up
-	2101608040029452931_15_49, 2, medium close up
-	2101608040029452931_15_49, 3, medium close up
-	2101608040029452931_15_50, 0, medium close up

In [125]:
cq13 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    
    SELECT DISTINCT ?emo_ann ?shot ?shottype 

    WHERE{
        ?emo_ann :framedBy ?shot;
                :be_emotion :anger.
        ?shot :hasShotType  ?shottype.

        }
"""

results = modelet1.query(cq13)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_49'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_49_utterance_0'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#close_up'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_49'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_49_utterance_1'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#medium_close_up'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_49'), rdflib.term.URIRef('https://raw.githubusercontent.com/Madd

#### 14. What type of visual cues are shown in medium close ups where enjoyment is expressed?

Expected results:

A list of types of types of visual cues, the visual cues

Face Movement, smile

In [126]:
cq14 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    
    SELECT ?vis_type ?vis_cue

    WHERE{
        ?emo_ann :framedBy ?shot;
                :be_emotion :enjoyment.
        ?shot :hasShotType  :medium_close_up;
            dul:realizes ?vis_cue.
        ?vis_cue a ?vis_type.
        ?vis_type  rdfs:subClassOf :HumanVisualContent.        
        }
"""

results = modelet1.query(cq14)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#FaceMovement'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frown'))




#### 15. What facial expressions are shown in shots of utterances where sadness is expressed by the experiencer?

Expected result:

A list of face movements.

* close eyes 
* frown
* widen eyes 
* stretch mouth
* raise inner brow

In [128]:
cq15 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    
    SELECT DISTINCT ?face_mov

    WHERE{
        ?emo_ann :framedBy ?shot;
                :be_emotion :sadness.
        ?shot dul:realizes ?face_mov.
        ?face_mov a :FaceMovement.
        
        }
"""

results = modelet1.query(cq15)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#close_eyes'),)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#frown'),)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#raise_inner_brow'),)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#stretch_mouth'),)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#widen_eyes'),)




#### 16. What utterances have frames with voiceover? 
Expected result:

A list of utterances

-   2101608040029452931_15_43
-	2101608040029452931_15_44
-	2101608040029452931_15_47
-	2101608040029452931_15_49


In [130]:
cq16 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    
    SELECT DISTINCT ?emo_ann

    WHERE{
        ?emo_ann :framedBy ?shot.
        ?shot :hasAuralComposition :voiceover.       
        }
"""

results = modelet1.query(cq16)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_43'),)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_44'),)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_47'),)


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_49'),)




#### 17. What is shown in utterances where speech is not aligned with the video? What is their aural composition? With what type of shot are they framed?

Expected result:

The utterance id, the aural composition, visual triggers 



In [133]:
cq17 = """
    PREFIX : <https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#> 
    PREFIX dul: <http://www.loa-cnr.it/ontologies/DUL.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    
    SELECT DISTINCT ?emo_ann ?vt

    WHERE{
        ?emo_ann :framedBy ?shot.
        
        OPTIONAL{
        ?emo_ann :hasVisualTrigger ?vt
        }
        
        ?shot :hasAuralComposition :speech_overlap. 
        }
"""

results = modelet1.query(cq17)

for row in results:
    print(row)
    print("\n")

(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_46'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#hand_to_mouth'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_46'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#raise_finger'))


(rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#2101608040029452931_15_46'), rdflib.term.URIRef('https://raw.githubusercontent.com/MaddaGh/DataProcessing_EmoAnnotation/main/meaw.ttl#smile'))


